**1.Data Generating**

In [ ]:
from dataIO import pk
import cv2 as cv
import numpy as np
import os

from numpy import random
import matplotlib.pyplot as plt
import scipy.misc
from skimage.io import imread
from glob import glob

from keras.layers import Conv2D, BatchNormalization, Activation,Merge, merge
from keras.models import Model, Input
import keras.backend as K
import tensorflow as tf
import gc

from __future__ import print_function
import os.path
import torch.utils.data as data
from torchvision import transforms
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from PIL import Image
import torchvision.models as models

from utils import Feline, sampleAccuracy, sampleLoss, VGGpredict,limit_mem

In [ ]:
height = 64
width = 64
imagedic = {"data":np.zeros(shape=[0,height,width,3],dtype = int),"label":[],"blur_data":np.zeros(shape=[0,height,width,3],dtype = int)}
rootdir = "./testimage"
pathlist = os.listdir(rootdir) #list all the file in path
kernel_size = (21, 21)
label = 0

In [ ]:
for i in range(len(pathlist)):
    label = pathlist[i]
    subpath = os.path.join(rootdir,pathlist[i])
    sublist = os.listdir(subpath)
    for j in range(len(sublist)):
        path = os.path.join(subpath,sublist[j])
        if os.path.isfile(path):
            img = cv.imread(path)
            res = cv.resize(img,(width, height), interpolation = cv.INTER_CUBIC)
            img_ = cv.blur(img, kernel_size)
            res_ = cv.resize(img_,(width, height), interpolation = cv.INTER_CUBIC)
            imagedic['data'] = np.append(imagedic['data'],[res], axis = 0).astype(np.uint8)
            imagedic['label'].append(int(label))
            imagedic['blur_data'] = np.append(imagedic['blur_data'],[res_], axis = 0).astype(np.uint8)

**2.Image Deblurring**

In [ ]:
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.3
K.set_session(tf.Session(config=config))

In [ ]:
X_test = {}
X_test['blur_data']=imagedic['blur_data'].astype(float)
X_test['data']=imagedic['data'].astype(float)

In [ ]:
def plot_loss(train,valid):
    fig,ax = plt.subplots()     
    plt.xlabel('Training epoches')  
    plt.ylabel('Loss values')    
    x = range(len(train))

    ax.plot(x,train,'g',label = 'train set')
    ax.plot(x,valid,'r',label = 'validation set')
    plt.grid(True)  
    plt.legend(bbox_to_anchor=(1.0, 1), loc=1, borderaxespad=0.)  
    plt.show()
    plt.pause(0.001)

In [ ]:
deblur_CNN_input = Input(shape=(64,64,3))

#HIDDEN LAYERS
deblur_CNN_layer1 = Conv2D(filters=128, kernel_size=10, strides = 1, padding='same')(deblur_CNN_input)
deblur_CNN_layer1 = BatchNormalization()(deblur_CNN_layer1)
deblur_CNN_layer1 = Activation('relu')(deblur_CNN_layer1)

deblur_CNN_layer2 = Conv2D(filters=320, kernel_size=1, strides = 1, padding='same')(deblur_CNN_layer1)
deblur_CNN_layer2 = BatchNormalization()(deblur_CNN_layer2)
deblur_CNN_layer2 = Activation('relu')(deblur_CNN_layer2)

deblur_CNN_layer3 = Conv2D(filters=320, kernel_size=1, strides = 1, padding='same')(deblur_CNN_layer2)
deblur_CNN_layer3= BatchNormalization()(deblur_CNN_layer3)
deblur_CNN_layer3 = Activation('relu')(deblur_CNN_layer3)

deblur_CNN_layer4 = Conv2D(filters=320, kernel_size=1, strides = 1, padding='same')(deblur_CNN_layer3)
deblur_CNN_layer4 = BatchNormalization()(deblur_CNN_layer4)
deblur_CNN_layer4 = Activation('relu')(deblur_CNN_layer4)

deblur_CNN_layer5 = Conv2D(filters=128, kernel_size=1, strides = 1, padding='same')(deblur_CNN_layer4)
deblur_CNN_layer5 = BatchNormalization()(deblur_CNN_layer5)
deblur_CNN_layer5 = Activation('relu')(deblur_CNN_layer5)

deblur_CNN_layer6 = Conv2D(filters=128, kernel_size=3, strides = 1, padding='same')(deblur_CNN_layer5)
deblur_CNN_layer6 = BatchNormalization()(deblur_CNN_layer6)
deblur_CNN_layer6 = Activation('relu')(deblur_CNN_layer6)

deblur_CNN_layer7 = Conv2D(filters=512, kernel_size=1, strides = 1, padding='same')(deblur_CNN_layer6)
deblur_CNN_layer7 = BatchNormalization()(deblur_CNN_layer7)
deblur_CNN_layer7 = Activation('relu')(deblur_CNN_layer7)

deblur_CNN_layer8 = Conv2D(filters=128, kernel_size=5, strides = 1, padding='same')(deblur_CNN_layer7)
deblur_CNN_layer8 = BatchNormalization()(deblur_CNN_layer8)
deblur_CNN_layer8 = Activation('relu')(deblur_CNN_layer8)

deblur_CNN_layer9 = Conv2D(filters=128, kernel_size=5, strides = 1, padding='same')(deblur_CNN_layer8)
deblur_CNN_layer9 = BatchNormalization()(deblur_CNN_layer9)
deblur_CNN_layer9 = Activation('relu')(deblur_CNN_layer9)

deblur_CNN_layer10 = Conv2D(filters=128, kernel_size=3, strides = 1, padding='same')(deblur_CNN_layer9)
deblur_CNN_layer10 = BatchNormalization()(deblur_CNN_layer10)
deblur_CNN_layer10 = Activation('relu')(deblur_CNN_layer10)

deblur_CNN_layer11 = Conv2D(filters=128, kernel_size=5, strides = 1, padding='same')(deblur_CNN_layer10)
deblur_CNN_layer11 = BatchNormalization()(deblur_CNN_layer11)
deblur_CNN_layer11 = Activation('relu')(deblur_CNN_layer11)

deblur_CNN_layer12 = Conv2D(filters=128, kernel_size=5, strides = 1, padding='same')(deblur_CNN_layer11)
deblur_CNN_layer12 = BatchNormalization()(deblur_CNN_layer12)
deblur_CNN_layer12 = Activation('relu')(deblur_CNN_layer12)

deblur_CNN_layer13 = Conv2D(filters=256, kernel_size=1, strides = 1, padding='same')(deblur_CNN_layer12)
deblur_CNN_layer13 = BatchNormalization()(deblur_CNN_layer13)
deblur_CNN_layer13 = Activation('relu')(deblur_CNN_layer13)

deblur_CNN_layer14 = Conv2D(filters=64, kernel_size=7, strides = 1, padding='same')(deblur_CNN_layer13)
deblur_CNN_layer14 = BatchNormalization()(deblur_CNN_layer14)
deblur_CNN_layer14 = Activation('relu')(deblur_CNN_layer14)

deblur_CNN_output = Conv2D(filters=3, kernel_size=7, strides = 1, padding='same', activation='relu')(deblur_CNN_layer14)

deblur_CNN = Model(inputs= deblur_CNN_input, outputs=deblur_CNN_output )

deblur_CNN.load_weights('weights_de_merge_10.h5')

deblur_CNN.summary()

In [ ]:
Deblurred = deblur_CNN.predict(X_test['blur_data'])
Deblurred = np.clip(Deblurred, 0, 255)
Deblurred=Deblurred.astype(np.uint8)
f, ax = plt.subplots(3,10, figsize=(15,5))
for i in range(10):
    ax[0,i].imshow(cv.cvtColor(X_test['data'][i].astype('uint8'),cv.COLOR_BGR2RGB));  ax[0,i].axis('Off'); ax[0,i].set_title('Clean', size=15)
    ax[1,i].imshow(cv.cvtColor(X_test['blur_data'][i].astype('uint8'),cv.COLOR_BGR2RGB));  ax[1,i].axis('Off'); ax[1,i].set_title('Blurry', size=15)
    ax[2,i].imshow(cv.cvtColor(Deblurred[i].astype('uint8'),cv.COLOR_BGR2RGB));  ax[2,i].axis('Off'); ax[2,i].set_title('Deblurred', size=15)
plt.show()

In [ ]:
imagedic['deblur_data'] = Deblurred

**3.Classifying(VGG16)**

In [ ]:
class VGG_16(nn.Module):
    def __init__(self):
        super(VGG_16, self).__init__()
        
        self.vgg = models.vgg16(pretrained=True)        
        self.vgg.classifier = nn.Sequential(
            *(self.vgg.classifier[i] for i in range(6)))
        self.vgg.classifier.add_module('fc',nn.Linear(4096,4))
        
    def forward(self, images):        
        return self.vgg(images)

In [ ]:
vggmodel = torch.load('vgg16_2.pt')
vggmodel.eval()

In [ ]:
Transform = transforms.Compose([transforms.Resize((224,224)),transforms.ToTensor()])
Origin_test = Feline(imagedic, Deblur=False, Blured=False, transform=Transform)#transforms.ToTensor()
Blurred_test = Feline(imagedic, Deblur=False, Blured=True, transform=Transform)#transforms.ToTensor()
Deblur_test = Feline(imagedic, Deblur=True, Blured=False, transform=Transform)#transforms.ToTensor()
Origin_test_loader = torch.utils.data.DataLoader(Origin_test, batch_size=1, shuffle=True)
Blurred_test_loader = torch.utils.data.DataLoader(Blurred_test, batch_size=1, shuffle=True)
Deblur_test_loader = torch.utils.data.DataLoader(Deblur_test, batch_size=1, shuffle=True)

In [ ]:
Origin_predict = VGGpredict(Origin_test_loader, vggmodel)
Blurred_predict = VGGpredict(Blurred_test_loader, vggmodel)
Deblur_predict = VGGpredict(Deblur_test_loader, vggmodel)

In [ ]:
feline_class = ['cat','tiger','lion','cheetah']
for i in range(len(Origin_predict)):
    plt.figure()
    f, ax = plt.subplots(1,3, figsize=(8,3))
    print('class: Ground truth: {}; Origin image: {}; Blurred image: {}; Deblurred image: {}'\
          .format(feline_class[imagedic['label'][i]], feline_class[Origin_predict[i]],\
                feline_class[Blurred_predict[i]], feline_class[Deblur_predict[i]]))
    ax[0].imshow(cv.cvtColor(imagedic['data'][i].astype('uint8'),cv.COLOR_BGR2RGB));  ax[0].axis('Off'); ax[0].set_title('Origin', size=15)
    ax[1].imshow(cv.cvtColor(imagedic['blur_data'][i].astype('uint8'),cv.COLOR_BGR2RGB));  ax[1].axis('Off'); ax[1].set_title('Blurred', size=15)
    ax[2].imshow(cv.cvtColor(imagedic['deblur_data'][i].astype('uint8'),cv.COLOR_BGR2RGB));  ax[2].axis('Off'); ax[2].set_title('Deblurred', size=15)
    plt.pause(0.1)